In [1]:
import numpy as np
import pandas as pd
import sqlite3

In [2]:
SQL_DB = "lahmansbaseballdb.sqlite"  # local file copy of Lahman's DB, as done in .sqlite by ____(github reference)

Find out the table names for the above DB

In [3]:
query = "SELECT name FROM sqlite_master WHERE type='table'"

with sqlite3.connect(SQL_DB) as connection:
    tables = pd.read_sql_query(query, connection)

In [11]:
tables.name.values

array(['allstarfull', 'appearances', 'awardsmanagers', 'awardsplayers',
       'awardssharemanagers', 'awardsshareplayers', 'batting',
       'battingpost', 'collegeplaying', 'divisions', 'fielding',
       'fieldingof', 'fieldingofsplit', 'fieldingpost', 'halloffame',
       'homegames', 'leagues', 'managers', 'managershalf', 'parks',
       'people', 'pitching', 'pitchingpost', 'salaries', 'schools',
       'seriespost', 'teams', 'teamsfranchises', 'teamshalf'],
      dtype=object)

The `people` table looks like the one with details for individual players

In [13]:
query = "SELECT * FROM people LIMIT 5;"

with sqlite3.connect(SQL_DB) as connection:
    people_head = pd.read_sql_query(query, connection)
    
people_head

,playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,...,bats,throws,debut,finalGame,retroID,bbrefID,birth_date,debut_date,finalgame_date,death_date
0,aardsda01,1981,12,27,USA,CO,Denver,NaN,NaN,NaN,...,R,R,2004-04-06,2015-08-23,aardd001,aardsda01,1981-12-27,2004-04-06,2015-08-23,None
1,aaronha01,1934,2,5,USA,AL,Mobile,NaN,NaN,NaN,...,R,R,1954-04-13,1976-10-03,aaroh101,aaronha01,1934-02-05,1954-04-13,1976-10-03,None
2,aaronto01,1939,8,5,USA,AL,Mobile,1984.0,8.0,16.0,...,R,R,1962-04-10,1971-09-26,aarot101,aaronto01,1939-08-05,1962-04-10,1971-09-26,1984-08-16
3,aasedo01,1954,9,8,USA,CA,Orange,NaN,NaN,NaN,...,R,R,1977-07-26,1990-10-03,aased001,aasedo01,1954-09-08,1977-07-26,1990-10-03,None
4,abadan01,1972,8,25,USA,FL,Palm Beach,NaN,NaN,NaN,...,L,L,2001-09-10,2006-04-13,abada001,abadan01,1972-08-25,2001-09-10,2006-04-13,None


`playerID` looks same as `bbrefID`, but I'll keep `bbrefID` since I can get per-game stats on baseball-reference.com

In [18]:
# only going to focus on last 100 years
query = "SELECT bbrefID, debut, finalGame \
            FROM people \
            WHERE finalGame >= '1920-01-01';"

with sqlite3.connect(SQL_DB) as connection:
    modern_players = pd.read_sql_query(query, connection)
    
print(modern_players.shape)
print()
modern_players.head()

(15332, 3)



,bbrefID,debut,finalGame
0,aardsda01,2004-04-06,2015-08-23
1,aaronha01,1954-04-13,1976-10-03
2,aaronto01,1962-04-10,1971-09-26
3,aasedo01,1977-07-26,1990-10-03
4,abadan01,2001-09-10,2006-04-13


In [19]:
# check that nothing went wrong with the date filtering, which it usually does....
print(modern_players.finalGame.min(), modern_players.finalGame.max())

1920-04-18 2019-09-29


Now, with those 15K player ID's and dates, you can go to baseball-reference.com and get individual game stats for each player.

In [22]:
# sample of a player/season/allgames URL from the reference:
# This is Byron Buxton's 2019 table, read in as a list of pd.DataFrames:
buxton19 = pd.read_html('https://www.baseball-reference.com/players/gl.fcgi?id=buxtoby01&t=b&year=2019', index_col=0)
type(buxton19)

list

In [23]:
[type(x) for x in buxton19]

[pandas.core.frame.DataFrame,
 pandas.core.frame.DataFrame,
 pandas.core.frame.DataFrame,
 pandas.core.frame.DataFrame,
 pandas.core.frame.DataFrame]

In [24]:
for frame in buxton19:
    print(frame.head(3))
    print()

                                                   1
0                                                   
Team Record in Appearances  62-25 / in Starts: 53-25
Longest Hitting Streak                       6 games
Longest On Base Streak                      11 games

                                      1
0                                      
in 295 PAs                           46
Actual Runners on Base  195 (105-61-29)
Avg. MLBer w/ 295 PAs                36

           1
0           
Self      10
M Sano     8
J Schoop   8

            1
0            
Self       10
M Kepler    9
J Polanco   7

   Gcar Gtm    Date   Tm Unnamed: 5  Opp   Rslt Inngs PA AB  ...   OBP   SLG  \
Rk                                                           ...               
1   307   1  Mar 28  MIN        NaN  CLE  W,2-0    CG  3  3  ...  .333  .667   
2   308   2  Mar 30  MIN        NaN  CLE  L,1-2    CG  3  3  ...  .333  .667   
3   309   3  Mar 31  MIN        NaN  CLE  W,9-3    CG  4  4  ...  .400  .700

So the last element in that list has the target data.

In [35]:
def get_player_year(playerID, year):
    base_url = 'https://www.baseball-reference.com/players/gl.fcgi?id='
    full_url = base_url + playerID + '&t=b&year=' + str(year)
    try:
        return pd.read_html(full_url, index_col=0)[-1]  # the last DF in the list
    except:
        return None  

15K players, maybe 8 seasons each, maybe 100 games a year, that's 12M rows.  Could make one big DF but since the goal is to look at single seasons, could also make 100 DF's, one per year.  I'd like to start with one big DF and trim out the games I'm not interested in.

In [68]:
modern_players.set_index('bbrefID', inplace=True)
modern_players.head(3)

,debut,finalGame
bbrefID,,
aardsda01,2004-04-06,2015-08-23
aaronha01,1954-04-13,1976-10-03
aaronto01,1962-04-10,1971-09-26


In [73]:
player_dict = modern_players.to_dict('index')
player_dict = {p:(int(player_dict[p]['debut'][:4]), 
                  int(player_dict[p]['finalGame'][:4])) 
                  for p in player_dict}
player_dict['aaronha01']

(1954, 1976)

In [74]:
modern_players.iloc[0]

debut        2004-04-06
finalGame    2015-08-23
Name: aardsda01, dtype: object

In [ ]:
from tqdm import tqdm
player_years = dict()

def get_player_batch(start, end):
    for player in tqdm(modern_players.index[start:end]):
        years = []
        ## THE NEXT LINE SHOULD BE RANGE(PLAYER_DICT[PLAYER][0], PLAYER_DICT[PLAYER][1] + 1)
        for yr in range(*player_dict[player]):
            years.append((yr, get_player_year(player, yr)))
        player_years[player] = years
        
get_player_batch(0,10)

In [82]:
len(player_years)

10

In [79]:
print(f"Hank Aaron's first season ({player_years['aaronha01'][0][0]}):")
player_years['aaronha01'][0][1] # hank aaron's first season

Hank Aaron's first season (1954):


,Gcar,Gtm,Date,Tm,Unnamed: 5,Opp,Rslt,Inngs,PA,AB,...,CS,BA,OBP,SLG,OPS,BOP,aLI,WPA,RE24,Pos
Rk,,,,,,,,,,,,,,,,,,,,,
1,1,1,Apr 13,MLN,@,CIN,"L,8-9",CG,5,5,...,0,.000,.000,.000,.000,5,1.12,-0.188,-1.96,LF
2,2,2,Apr 15,MLN,NaN,STL,"W,7-6",CG(11),5,5,...,0,.200,.200,.300,.500,6,.63,-0.036,-0.27,LF
3,3,3,Apr 17,MLN,NaN,CIN,"W,5-1",CG,4,2,...,0,.250,.357,.417,.774,6,.51,0.052,1.15,LF
4,4,4,Apr 18 (1),MLN,NaN,CIN,"L,4-6",CG,4,3,...,0,.200,.333,.333,.667,6,.62,-0.046,-0.42,RF
5,5,5,Apr 18 (2),MLN,NaN,CIN,"L,3-5",CG,4,4,...,0,.211,.318,.316,.634,6,1.40,-0.131,-0.72,RF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Sept,Sept,Gtm,Date,Tm,NaN,Opp,Rslt,Inngs,PA,AB,...,CS,BA,OBP,SLG,OPS,BOP,aLI,WPA,RE24,Pos
120,120,131 (3),Sep 4,MLN,@,CIN,"W,5-4",7-GF(11),2,2,...,0,.272,.314,.432,.746,4,1.61,-0.083,-0.35,LF
121,121,132,Sep 5 (1),MLN,@,CIN,"W,11-8",7-GF,1,1,...,0,.274,.315,.435,.751,4,.10,0.007,0.42,PR LF


In [83]:
get_player_batch(10,100)

100%|██████████| 90/90 [02:17<00:00,  1.53s/it]


In [85]:
get_player_batch(100, 1000)

100%|██████████| 900/900 [28:51<00:00,  1.92s/it]  


In [94]:
get_player_batch(1000, len(modern_players))

100%|██████████| 14332/14332 [6:48:54<00:00,  1.71s/it]  


In [95]:
import pickle

7 or 8 hours to pull down every game for every player for 100 years.  Should probably save the data to avoid repeating.

In [96]:
with open('player_game_dict.pkl', 'wb') as f:
    pickle.dump(player_years, f)

In [97]:
!ls -l player_game_dict.pkl

-rw-r--r--  1 ethanhaley  staff  1270893749 Sep  8 06:29 player_game_dict.pkl


In [98]:
player_years['aaronha01'][0][1].index

Index([   '1',    '2',    '3',    '4',    '5',    '6',    '7',    '8',    '9',
         '10',
       ...
        '115',  '116',  '117',  '118',  '119', 'Sept',  '120',  '121',  '122',
          nan],
      dtype='object', name='Rk', length=128)

128 rows for Aaron's first season, for 122 games.  The extra 6 rows are for month name dividers and for summaries.

In [99]:
player_years['aaronha01'][0][1].columns

Index(['Gcar', 'Gtm', 'Date', 'Tm', 'Unnamed: 5', 'Opp', 'Rslt', 'Inngs', 'PA',
       'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'BB', 'IBB', 'SO', 'HBP', 'SH',
       'SF', 'ROE', 'GDP', 'SB', 'CS', 'BA', 'OBP', 'SLG', 'OPS', 'BOP', 'aLI',
       'WPA', 'RE24', 'Pos'],
      dtype='object')

The 'Gtm' column is the game of the season for the team, which will be important for this project.  Keep everything for now.  To paste together all these data frames sensibly, we also need to add the player and the year as columns for each frame.  Might as well make them the indices, along with 'Gtm'.

In [352]:
def process_career(player, p_dict):
    '''Clean up the input frame by removing header and footer rows,
    so that it can be pasted together with other player career frames.
    Create a multi-index [playerID, year, game#(of team)] that will
    uniquely identify each row of a large frame, 100 years of MLB.
    Return the player's entire career as a new frame, with one row
    for each game he played.  Use these career frames to build the MLB frame.
    '''
    seasons = []
    for (season, stats) in p_dict[player].copy():
        if stats is None: continue # for players who didn't play every season
        try:
            stats = stats.loc[stats.index.dropna()]
            is_game = [str(i)[0].isnumeric() for i in stats.index] # first char only for str/float/int
            stats = stats[is_game]
            # add indices
            stats['player'] = [player] * len(stats)
            stats['season'] = [season] * len(stats)
            stats.set_index(['player', 'season', 'Gtm'], inplace=True)
            seasons.append(stats)
        except AttributeError:
            print(player, season)
    return pd.concat(seasons) if seasons else None

In [326]:
player_years['buxtoby01']

[(2015,             Gcar       Gtm        Date   Tm Unnamed: 5  Opp    Rslt  \
  Rk                                                                    
  1              1   62 (61)      Jun 14  MIN          @  TEX   W,4-3   
  2              2        63      Jun 15  MIN          @  STL   L,2-3   
  3              3        64      Jun 16  MIN          @  STL   L,2-3   
  4              4        65      Jun 17  MIN        NaN  STL   W,3-1   
  5              5        66      Jun 18  MIN        NaN  STL   W,2-1   
  6              6        67      Jun 19  MIN        NaN  CHC   W,7-2   
  7              7        68      Jun 20  MIN        NaN  CHC   L,1-4   
  8              8        69      Jun 21  MIN        NaN  CHC   L,0-8   
  9              9        70      Jun 22  MIN        NaN  CHW  W,13-2   
  10            10        71      Jun 23  MIN        NaN  CHW   L,2-6   
  11            11        72      Jun 24  MIN        NaN  CHW   W,6-1   
  Aug          Aug       Gtm        Date   Tm

In [208]:
player_gen = (p for p in player_years)
all_p = process_career(next(player_gen), player_years)
all_p

Gcar    Date   Tm Unnamed: 5  Opp   Rslt      Inngs  \
player    season Gtm                                                           
aardsda01 2004   2 (1)      1   Apr 6  SFG          @  HOU  W,7-5        6-8   
                 4 (1)      2   Apr 8  SFG          @  SDP  L,3-4  10-GF(10)   
                 5          3  Apr 10  SFG          @  SDP  L,4-6        7-7   
                 9 (3)      4  Apr 14  SFG        NaN  MIL  L,0-3        7-7   
                 11 (1)     5  Apr 17  SFG        NaN  LAD  L,4-5        7-8   
...                       ...     ...  ...        ...  ...    ...        ...   
          2013   150 (2)  105  Sep 17  NYM        NaN  SFG  L,5-8        9-9   
                 151      106  Sep 18  NYM        NaN  SFG  W,5-4        8-8   
                 156 (4)  107  Sep 23  NYM          @  CIN  L,2-3       9-10   
                 159 (2)  108  Sep 26  NYM        NaN  MIL  L,2-4        8-9   
                 161 (1)  109  Sep 28  NYM        NaN  MIL  L,2-4        7-7   

                         PA AB  R  ... CS   BA  OBP  SLG  OPS BOP  aLI  WPA  \
player    season Gtm               ...                                        
aardsda01 2004   2 (1)    0  0  0  ...  0  NaN  NaN  NaN  NaN   9  NaN  NaN   
                 4 (1)    0  0  0  ...  0  NaN  NaN  NaN  NaN   9  NaN  NaN   
                 5        0  0  0  ...  0  NaN  NaN  NaN  NaN   9  NaN  NaN   
                 9 (3)    0  0  0  ...  0  NaN  NaN  NaN  NaN   9  NaN  NaN   
                 11 (1)   0  0  0  ...  0  NaN  NaN  NaN  NaN   7  NaN  NaN   
...                      .. .. ..  ... ..  ...  ...  ...  ...  ..  ...  ...   
          2013   150 (2)  0  0  0  ...  0  NaN  NaN  NaN  NaN   5  NaN  NaN   
                 151      0  0  0  ...  0  NaN  NaN  NaN  NaN   9  NaN  NaN   
                 156 (4)  0  0  0  ...  0  NaN  NaN  NaN  NaN   9  NaN  NaN   
                 159 (2)  0  0  0  ...  0  NaN  NaN  NaN  NaN   9  NaN  NaN   
                 161 (1)  0  0  0  ...  0  NaN  NaN  NaN  NaN   9  NaN  NaN   

                         RE24 Pos  
player    season Gtm               
aardsda01 2004   2 (1)    NaN   P  
                 4 (1)    NaN   P  
                 5        NaN   P  
                 9 (3)    NaN   P  
                 11 (1)   NaN   P  
...                       ...  ..  
          2013   150 (2)  NaN   P  
                 151      NaN   P  
                 156 (4)  NaN   P  
                 159 (2)  NaN   P  
                 161 (1)  NaN   P  

[109 rows x 34 columns]

In [226]:
for _ in tqdm(range(1000)):
    all_p = pd.concat([all_p, process_career(next(player_gen), player_years)], sort=True)

  0%|          | 1/1000 [00:00<02:27,  6.79it/s]/Users/ethanhaley/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

100%|██████████| 1000/1000 [14:22<00:00,  1.16it/s]


In [227]:
all_p.shape

(294556, 41)

In [228]:
all_p.tail()

2B 3B AB    BA BB BOP CS DFS(DK) DFS(FD)  \
player    season Gtm                                                  
birdgr01  2018   151 (4)    0  0  1  .195  0   2  0    2.00    3.50   
                 158 (6)    0  0  0  .195  0   8  0    0.00    0.00   
                 159        0  0  1  .195  0   2  0    0.00    0.00   
                 161 (1)    1  0  5  .199  0   7  0   12.00   15.70   
birkbmi01 1992   129 (128)  0  0  2     0  0   9  0     NaN     NaN   

                                  Date  ... SO   Tm Unnamed: 33 Unnamed: 34  \
player    season Gtm                    ...                                   
birdgr01  2018   151 (4)        Sep 19  ...  0  NYY         NaN         NaN   
                 158 (6)        Sep 26  ...  0  NYY         NaN         NaN   
                 159            Sep 27  ...  0  NYY         NaN         NaN   
                 161 (1)        Sep 29  ...  2  NYY         NaN         NaN   
birkbmi01 1992   129 (128)  Aug 31 (2)  ...  0  NYM         NaN         NaN   

                           Unnamed: 35 Unnamed: 36 Unnamed: 37 Unnamed: 5  \
player    season Gtm                                                        
birdgr01  2018   151 (4)           NaN         NaN         NaN        NaN   
                 158 (6)           NaN         NaN         NaN          @   
                 159               NaN         NaN         NaN          @   
                 161 (1)           NaN         NaN         NaN          @   
birkbmi01 1992   129 (128)         NaN         NaN         NaN        NaN   

                              WPA   aLI  
player    season Gtm                     
birdgr01  2018   151 (4)    0.000   .00  
                 158 (6)      NaN   NaN  
                 159        0.000   .00  
                 161 (1)    0.108   .51  
birkbmi01 1992   129 (128) -0.059  1.09  

[5 rows x 41 columns]

Seems like batches would make this more efficient (appending each player's career one at a time took almost 1 sec per player for the first 1000 players).

In [301]:
mainframe = process_career(next(player_gen), player_years)

for _ in tqdm(range(10)):
    batches = []
    for _ in tqdm(range(100)):
        batches.append(process_career(next(player_gen), player_years))
    batches = pd.concat(batches)
    mainframe = pd.concat([mainframe, batches])

  0%|          | 0/100 [00:00<?, ?it/s]/Users/ethanhaley/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.


100%|██████████| 100/100 [00:03<00:00, 33.09it/s]
/Users/ethanhaley/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  import sys
 61%|██████    | 61/100 [00:01<00:01, 32.92it/s]


100%|██████████| 10/10 [00:40<00:00,  4.07s/it]


In [302]:
mainframe.shape

(271510, 41)

Yes, that was about 50x faster than without batches

In [303]:
all_p = pd.concat([all_p, mainframe])

In [304]:
all_p.shape

(3974291, 41)

In [305]:
all_p.tail()

2B 3B AB    BA BB BOP CS DFS(DK) DFS(FD)        Date  \
player    season Gtm                                                            
wrighja02 2004   153 (3)  0  0  2  .105  0   9  0     NaN     NaN      Sep 23   
                 158 (4)  0  0  0  .105  0   9  0     NaN     NaN  Sep 28 (2)   
          2006   65 (64)  0  0  2  .000  1   9  0     NaN     NaN      Jun 16   
                 70 (4)   0  0  1  .000  0   9  0     NaN     NaN      Jun 21   
                 95 (24)  0  0  0  .000  0   9  0     NaN     NaN      Jul 22   

                          ... SO   Tm Unnamed: 33 Unnamed: 34 Unnamed: 35  \
player    season Gtm      ...                                               
wrighja02 2004   153 (3)  ...  2  ATL         NaN         NaN         NaN   
                 158 (4)  ...  0  ATL         NaN         NaN         NaN   
          2006   65 (64)  ...  1  NYY         NaN         NaN         NaN   
                 70 (4)   ...  0  NYY         NaN         NaN         NaN   
                 95 (24)  ...  0  NYY         NaN         NaN         NaN   

                         Unnamed: 36 Unnamed: 37 Unnamed: 5     WPA   aLI  
player    season Gtm                                                       
wrighja02 2004   153 (3)         NaN         NaN        NaN  -0.075  1.40  
                 158 (4)         NaN         NaN        NaN     NaN   NaN  
          2006   65 (64)         NaN         NaN          @  -0.035  1.87  
                 70 (4)          NaN         NaN          @  -0.040  1.14  
                 95 (24)         NaN         NaN          @     NaN   NaN  

[5 rows x 41 columns]

Getting kind of close to end of generator there. Copy generator before each batch for a smooth landing at the end.

In [306]:
# easiest way to get the length of the items left in a generator is to exhaust it and rebuild it
gens = [p for p in player_gen]
len(gens)

196

In [307]:
player_gen = (p for p in gens)
type(player_gen)

generator

In [308]:
# 196 players left now
batches = []
for _ in tqdm(range(196)):
    batches.append(process_career(next(player_gen), player_years))
batches = pd.concat(batches)
batches.shape

  0%|          | 0/196 [00:00<?, ?it/s]/Users/ethanhaley/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

100%|██████████| 196/196 [00:04<00:00, 41.14it/s]
/Users/ethanhaley/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """


(59181, 41)

In [309]:
all_p = pd.concat([all_p, batches])
all_p.tail()

2B 3B AB    BA BB BOP CS DFS(DK) DFS(FD)        Date  \
player    season Gtm                                                            
zuverge01 1958   136 (1)  0  0  0  .222  0   5  0     NaN     NaN       Sep 7   
                 137      0  0  0  .222  0   9  0     NaN     NaN       Sep 9   
                 139 (1)  0  0  0  .222  0   9  0     NaN     NaN      Sep 11   
                 142 (2)  0  0  0  .222  0   9  0     NaN     NaN  Sep 14 (1)   
                 144 (1)  0  0  0  .222  0   9  0     NaN     NaN      Sep 16   

                          ... SO   Tm Unnamed: 33 Unnamed: 34 Unnamed: 35  \
player    season Gtm      ...                                               
zuverge01 1958   136 (1)  ...  0  BAL         NaN         NaN         NaN   
                 137      ...  0  BAL         NaN         NaN         NaN   
                 139 (1)  ...  0  BAL         NaN         NaN         NaN   
                 142 (2)  ...  0  BAL         NaN         NaN         NaN   
                 144 (1)  ...  0  BAL         NaN         NaN         NaN   

                         Unnamed: 36 Unnamed: 37 Unnamed: 5  WPA  aLI  
player    season Gtm                                                   
zuverge01 1958   136 (1)         NaN         NaN        NaN  NaN  NaN  
                 137             NaN         NaN          @  NaN  NaN  
                 139 (1)         NaN         NaN          @  NaN  NaN  
                 142 (2)         NaN         NaN          @  NaN  NaN  
                 144 (1)         NaN         NaN          @  NaN  NaN  

[5 rows x 41 columns]

In [310]:
all_p.shape

(4033472, 41)

So in the end, only 4M rows, not the estimated 12M, because these are batting statistics, and they include pitchers' stats at the plate.

===> I didn't pull in the last year for every player, due to messing up the range indexing 😩.  Will need to make 15K more calls to `pd.read_html(URL)`.

In [344]:
final_years = dict()

def get_last_year(start, end):
    for player in tqdm(modern_players.index[start:end]):
        year = player_dict[player][1]
        final_years[player] = (year, get_player_year(player, year))
        
        
get_last_year(0, len(modern_players.index))

100%|██████████| 15332/15332 [2:30:30<00:00,  1.70it/s]   


In [345]:
len(final_years)

15332

In [361]:
player_gen = (p for p in final_years)

In [357]:
def process_final_year(player, p_dict):
    
    season, stats = p_dict[player]
    if stats is None: return None # for players who didn't play every season
    try:
        stats = stats.loc[stats.index.dropna()]
        is_game = [str(i)[0].isnumeric() for i in stats.index] # first char only for str/float/int
        stats = stats[is_game]
        # add indices
        stats['player'] = [player] * len(stats)
        stats['season'] = [season] * len(stats)
        stats.set_index(['player', 'season', 'Gtm'], inplace=True)

    except AttributeError:
            print(player, season)
            
    return stats

In [390]:
mainframe = process_final_year(next(player_gen), final_years)

for _ in tqdm(range(10)):
    batches = []
    for _ in tqdm(range(100)):
        batches.append(process_final_year(next(player_gen), final_years))
    batches = pd.concat(batches)
    mainframe = pd.concat([mainframe, batches])

100%|██████████| 100/100 [00:00<00:00, 203.53it/s][A
/Users/ethanhaley/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  import sys
/Users/ethanhaley/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  
100%|██████████| 10/10 [00:10<00:00,  1.00s/it]


In [391]:
#all_finals = mainframe.copy()
all_finals = pd.concat([all_finals, mainframe])
all_finals.tail()

2B 3B AB     BA BB BOP CS DFS(DK) DFS(FD)  \
player   season Gtm                                                   
wisede01 2013   41         0  0  5   .230  1   1  0     NaN     NaN   
                47 (5)     1  0  1   .242  0   6  0     NaN     NaN   
                50 (2)     0  0  2   .234  0   9  0     NaN     NaN   
wisehu01 1930   151 (150)  0  0  2    0.5  0   8  0     NaN     NaN   
                153 (1)    0  0  4  0.333  0   8  0     NaN     NaN   

                                 Date  ...   SF SH    SLG SO   Tm Unnamed: 36  \
player   season Gtm                    ...                                      
wisede01 2013   41             May 18  ...    0  0   .311  1  CHW         NaN   
                47 (5)         May 25  ...    0  0   .339  0  CHW         NaN   
                50 (2)         May 29  ...    0  0   .328  1  CHW         NaN   
wisehu01 1930   151 (150)      Sep 26  ...  NaN  0    0.5  0  DET         NaN   
                153 (1)    Sep 27 (2)  ...  NaN  0  0.333  0  DET         NaN   

                          Unnamed: 37 Unnamed: 5     WPA   aLI  
player   season Gtm                                             
wisede01 2013   41                NaN          @  -0.157  1.58  
                47 (5)            NaN        NaN   0.173  2.26  
                50 (2)            NaN          @  -0.017   .35  
wisehu01 1930   151 (150)         NaN        NaN   0.005  0.52  
                153 (1)           NaN        NaN  -0.053  0.44  

[5 rows x 38 columns]

In [392]:
gens = [p for p in player_gen]
len(gens)

317

In [393]:
player_gen = (p for p in gens)
type(player_gen)

generator

In [394]:
# 317 players left now
batches = []
for _ in tqdm(range(317)):
    batches.append(process_final_year(next(player_gen), final_years))
batches = pd.concat(batches)
batches.shape

100%|██████████| 317/317 [00:01<00:00, 286.97it/s]
/Users/ethanhaley/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """


(7592, 38)

In [395]:
#all_finals = mainframe.copy()
all_finals = pd.concat([all_finals, batches])
all_finals.tail()

2B 3B AB   BA BB BOP CS DFS(DK) DFS(FD)    Date  ...  \
player    season Gtm                                                      ...   
zuverge01 1959   59 (1)   0  0  0  NaN  0   9  0     NaN     NaN  Jun 15  ...   
zychto01  2017   34 (33)  0  0  0  NaN  0   9  0     NaN     NaN  May 10  ...   
                 48 (13)  0  0  0  NaN  0   9  0     NaN     NaN  May 25  ...   
                 52 (3)   0  0  0  NaN  0   9  0     NaN     NaN  May 29  ...   
                 53       0  0  0  NaN  0   9  0     NaN     NaN  May 30  ...   

                           SF SH  SLG SO   Tm Unnamed: 36 Unnamed: 37  \
player    season Gtm                                                    
zuverge01 1959   59 (1)   NaN  0  NaN  0  BAL         NaN         NaN   
zychto01  2017   34 (33)  NaN  0  NaN  0  SEA         NaN         NaN   
                 48 (13)  NaN  0  NaN  0  SEA         NaN         NaN   
                 52 (3)   NaN  0  NaN  0  SEA         NaN         NaN   
                 53       NaN  0  NaN  0  SEA         NaN         NaN   

                         Unnamed: 5  WPA  aLI  
player    season Gtm                           
zuverge01 1959   59 (1)         NaN  NaN  NaN  
zychto01  2017   34 (33)          @  NaN  NaN  
                 48 (13)          @  NaN  NaN  
                 52 (3)           @  NaN  NaN  
                 53               @  NaN  NaN  

[5 rows x 38 columns]

In [396]:
all_p = pd.concat([all_p, all_finals])

/Users/ethanhaley/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [397]:
all_p.shape

(4421857, 41)

In [399]:
all_p.sort_index(inplace=True)

In [403]:
all_p.loc['buxtoby01']  # making sure last year(2019) is there now

2B 3B AB    BA BB BOP CS DFS(DK) DFS(FD)    Date  ... SO   Tm  \
season Gtm                                                        ...           
2015   121 (48)  0  0  6  .186  0   1  0     NaN     NaN  Aug 20  ...  1  MIN   
       122       0  0  5  .208  0   1  0     NaN     NaN  Aug 21  ...  1  MIN   
       123       1  0  4  .231  0   1  0     NaN     NaN  Aug 22  ...  1  MIN   
       124       0  0  6  .207  0   1  0     NaN     NaN  Aug 23  ...  2  MIN   
       125       1  0  4  .226  1   1  0     NaN     NaN  Aug 25  ...  2  MIN   
...             .. .. ..   ... ..  .. ..     ...     ...     ...  ... ..  ...   
2019   88        0  0  4  .253  0   9  0    7.00   10.00   Jul 6  ...  2  MIN   
       89        0  1  4  .253  0   9  0   15.00   18.50   Jul 7  ...  0  MIN   
       9         2  0  5  .292  0   8  0   19.00   24.40   Apr 9  ...  1  MIN   
       90        0  0  4  .253  0   9  0    8.00    9.00  Jul 12  ...  2  MIN   
       91        0  0  4  .249  0   9  0    0.00    0.00  Jul 13  ...  1  MIN   

                Unnamed: 33 Unnamed: 34 Unnamed: 35 Unnamed: 36 Unnamed: 37  \
season Gtm                                                                    
2015   121 (48)         NaN         NaN         NaN         NaN         NaN   
       122              NaN         NaN         NaN         NaN         NaN   
       123              NaN         NaN         NaN         NaN         NaN   
       124              NaN         NaN         NaN         NaN         NaN   
       125              NaN         NaN         NaN         NaN         NaN   
...                     ...         ...         ...         ...         ...   
2019   88               NaN         NaN         NaN         NaN         NaN   
       89               NaN         NaN         NaN         NaN         NaN   
       9                NaN         NaN         NaN         NaN         NaN   
       90               NaN         NaN         NaN         NaN         NaN   
       91               NaN         NaN         NaN         NaN         NaN   

                Unnamed: 5     WPA   aLI  
season Gtm                                
2015   121 (48)          @  -0.050   .28  
       122               @  -0.046  1.53  
       123               @   0.155  1.39  
       124               @  -0.154  1.00  
       125               @   0.084   .70  
...                    ...     ...   ...  
2019   88              NaN   0.106   .75  
       89              NaN  -0.041  1.65  
       9                 @   0.040   .48  
       90                @  -0.175  1.44  
       91                @  -0.070   .67  

[393 rows x 41 columns]

In [404]:
all_p.columns

Index(['2B', '3B', 'AB', 'BA', 'BB', 'BOP', 'CS', 'DFS(DK)', 'DFS(FD)', 'Date',
       'GDP', 'Gcar', 'H', 'HBP', 'HR', 'IBB', 'Inngs', 'OBP', 'OPS', 'Opp',
       'PA', 'Pos', 'R', 'RBI', 'RE24', 'ROE', 'Rslt', 'SB', 'SF', 'SH', 'SLG',
       'SO', 'Tm', 'Unnamed: 33', 'Unnamed: 34', 'Unnamed: 35', 'Unnamed: 36',
       'Unnamed: 37', 'Unnamed: 5', 'WPA', 'aLI'],
      dtype='object')

In [411]:
all_p['Unnamed: 5'].unique()

array([nan, '@'], dtype=object)

In [415]:
home = all_p['Unnamed: 5'].isna()

In [419]:
all_p["LOC"] = home.apply(lambda x: 'Home' if x else "Away")

In [422]:
all_p.drop(columns=['Unnamed: 5', 'Unnamed: 33', 'Unnamed: 34',
                    'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 37'], inplace=True)

In [424]:
pd.set_option('display.max_columns', None)

In [427]:
all_p.head().sort_index()

2B 3B AB   BA BB BOP CS DFS(DK) DFS(FD)    Date  \
player    season Gtm                                                        
aardsda01 2004   11 (1)    0  0  0  NaN  0   7  0     NaN     NaN  Apr 17   
                 12        0  0  0  NaN  0   7  0     NaN     NaN  Apr 18   
                 131 (48)  0  0  0  NaN  0   1  0     NaN     NaN  Aug 28   
                 14 (1)    0  0  0  NaN  0   9  0     NaN     NaN  Apr 20   
                 27 (12)   0  0  0  NaN  0   5  0     NaN     NaN   May 4   

                           GDP Gcar  H HBP HR  IBB Inngs  OBP  OPS  Opp PA  \
player    season Gtm                                                         
aardsda01 2004   11 (1)    NaN    5  0   0  0  NaN   7-8  NaN  NaN  LAD  0   
                 12        NaN    6  0   0  0  NaN   5-6  NaN  NaN  LAD  0   
                 131 (48)  NaN   11  0   0  0  NaN   7-9  NaN  NaN  ATL  0   
                 14 (1)    NaN    7  0   0  0  NaN   9-9  NaN  NaN  SDP  0   
                 27 (12)   NaN    8  0   0  0  NaN  8-GF  NaN  NaN  NYM  0   

                          Pos  R RBI RE24 ROE   Rslt SB   SF SH  SLG SO   Tm  \
player    season Gtm                                                           
aardsda01 2004   11 (1)     P  0   0  NaN   0  L,4-5  0  NaN  0  NaN  0  SFG   
                 12         P  0   0  NaN   0  L,6-7  0  NaN  0  NaN  0  SFG   
                 131 (48)   P  0   0  NaN   0  L,3-9  0  NaN  0  NaN  0  SFG   
                 14 (1)     P  0   0  NaN   0  L,5-9  0  NaN  0  NaN  0  SFG   
                 27 (12)    P  0   0  NaN   0  L,2-6  0  NaN  0  NaN  0  SFG   

                           WPA  aLI   LOC  
player    season Gtm                       
aardsda01 2004   11 (1)    NaN  NaN  Home  
                 12        NaN  NaN  Home  
                 131 (48)  NaN  NaN  Away  
                 14 (1)    NaN  NaN  Home  
                 27 (12)   NaN  NaN  Away

So the Gtm (team game number for the season) is not a number, because they wanted the parentheses to show how many games the player had sat out since his last game.  Therefore, '131 (48)' sorts as coming before '14'.  Need to make a column that just has the integer game number.

In [431]:
print(int('131 (48)'.split()[0]))
print(int('12'.split()[0]))

131
12


In [440]:
all_p.index[0][2].split()

['11', '(1)']

In [456]:
int(str(1.0).split()[0].split('.')[0])

1

In [457]:
game_numbers = all_p.index.map(lambda i: int(str(i[2]).split()[0].split('.')[0]))

By the way, the 'str()' part of the above lambda was because some rows have non-string floats for 'Gtm'.  Also, apply() doesn't work on multi indexes, but map() does.

In [466]:
# replace the Gtm level of the index with its integer equivalent
all_p.reset_index(level=2, inplace=True)
all_p['game'] = game_numbers
all_p.set_index('game', append=True, inplace=True)
all_p.head()

Gtm 2B 3B AB   BA BB BOP CS DFS(DK) DFS(FD)  \
player    season game                                                     
aardsda01 2004   11      11 (1)  0  0  0  NaN  0   7  0     NaN     NaN   
                 12          12  0  0  0  NaN  0   7  0     NaN     NaN   
                 131   131 (48)  0  0  0  NaN  0   1  0     NaN     NaN   
                 14      14 (1)  0  0  0  NaN  0   9  0     NaN     NaN   
                 27     27 (12)  0  0  0  NaN  0   5  0     NaN     NaN   

                         Date  GDP Gcar  H HBP HR  IBB Inngs  OBP  OPS  Opp  \
player    season game                                                         
aardsda01 2004   11    Apr 17  NaN    5  0   0  0  NaN   7-8  NaN  NaN  LAD   
                 12    Apr 18  NaN    6  0   0  0  NaN   5-6  NaN  NaN  LAD   
                 131   Aug 28  NaN   11  0   0  0  NaN   7-9  NaN  NaN  ATL   
                 14    Apr 20  NaN    7  0   0  0  NaN   9-9  NaN  NaN  SDP   
                 27     May 4  NaN    8  0   0  0  NaN  8-GF  NaN  NaN  NYM   

                      PA Pos  R RBI RE24 ROE   Rslt SB   SF SH  SLG SO   Tm  \
player    season game                                                         
aardsda01 2004   11    0   P  0   0  NaN   0  L,4-5  0  NaN  0  NaN  0  SFG   
                 12    0   P  0   0  NaN   0  L,6-7  0  NaN  0  NaN  0  SFG   
                 131   0   P  0   0  NaN   0  L,3-9  0  NaN  0  NaN  0  SFG   
                 14    0   P  0   0  NaN   0  L,5-9  0  NaN  0  NaN  0  SFG   
                 27    0   P  0   0  NaN   0  L,2-6  0  NaN  0  NaN  0  SFG   

                       WPA  aLI   LOC  
player    season game                  
aardsda01 2004   11    NaN  NaN  Home  
                 12    NaN  NaN  Home  
                 131   NaN  NaN  Away  
                 14    NaN  NaN  Home  
                 27    NaN  NaN  Away

In [469]:
# forgot to sort it
all_p.sort_index(inplace=True)

all_p.head(3)

Gtm 2B 3B AB   BA BB BOP CS DFS(DK) DFS(FD)    Date  \
player    season game                                                          
aardsda01 2004   2     2 (1)  0  0  0  NaN  0   9  0     NaN     NaN   Apr 6   
                 4     4 (1)  0  0  0  NaN  0   9  0     NaN     NaN   Apr 8   
                 5         5  0  0  0  NaN  0   9  0     NaN     NaN  Apr 10   

                       GDP Gcar  H HBP HR  IBB      Inngs  OBP  OPS  Opp PA  \
player    season game                                                         
aardsda01 2004   2     NaN    1  0   0  0  NaN        6-8  NaN  NaN  HOU  0   
                 4     NaN    2  0   0  0  NaN  10-GF(10)  NaN  NaN  SDP  0   
                 5     NaN    3  0   0  0  NaN        7-7  NaN  NaN  SDP  0   

                      Pos  R RBI RE24 ROE   Rslt SB   SF SH  SLG SO   Tm  WPA  \
player    season game                                                           
aardsda01 2004   2      P  0   0  NaN   0  W,7-5  0  NaN  0  NaN  0  SFG  NaN   
                 4      P  0   0  NaN   0  L,3-4  0  NaN  0  NaN  0  SFG  NaN   
                 5      P  0   0  NaN   0  L,4-6  0  NaN  0  NaN  0  SFG  NaN   

                       aLI   LOC  
player    season game             
aardsda01 2004   2     NaN  Away  
                 4     NaN  Away  
                 5     NaN  Away

Probably will want the actual names of these players at some point too.

In [473]:
query = "SELECT bbrefID, nameFirst, nameLast \
            FROM people \
            WHERE finalGame >= '1920-01-01';"

with sqlite3.connect(SQL_DB) as connection:
    modern_players = pd.read_sql_query(query, connection)
    
print(modern_players.shape)
print()
modern_players.head()

(15332, 3)



,bbrefID,nameFirst,nameLast
0,aardsda01,David,Aardsma
1,aaronha01,Hank,Aaron
2,aaronto01,Tommie,Aaron
3,aasedo01,Don,Aase
4,abadan01,Andy,Abad


In [492]:
head = modern_players.head().set_index('bbrefID')
hd = head.to_dict('index')
{id: ' '.join(hd[id].values()) for id in hd}

{'aardsda01': 'David Aardsma',
 'aaronha01': 'Hank Aaron',
 'aaronto01': 'Tommie Aaron',
 'aasedo01': 'Don Aase',
 'abadan01': 'Andy Abad'}

In [493]:
modern_players.set_index('bbrefID', inplace=True)
mp_dict = modern_players.to_dict('index')
mp_dict = {id: ' '.join(mp_dict[id].values()) for id in mp_dict}

In [494]:
all_p['name'] = all_p.index.map(lambda i: mp_dict[i[0]])

In [495]:
all_p.head(1)

,,,Gtm,2B,3B,AB,BA,BB,BOP,CS,DFS(DK),DFS(FD),Date,GDP,Gcar,H,HBP,HR,IBB,Inngs,OBP,OPS,Opp,PA,Pos,R,RBI,RE24,ROE,Rslt,SB,SF,SH,SLG,SO,Tm,WPA,aLI,LOC,name
player,season,game,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
aardsda01,2004,2,2 (1),0,0,0,NaN,0,9,0,NaN,NaN,Apr 6,NaN,1,0,0,0,NaN,6-8,NaN,NaN,HOU,0,P,0,0,NaN,0,"W,7-5",0,NaN,0,NaN,0,SFG,NaN,NaN,Away,David Aardsma


In [496]:
all_p.to_csv('century_batting')

In [ ]:
# if you want to have seasons as first sort instead:
## all_p = all_p.reorder_levels(['season', 'player', 'game'])

In [497]:
# if you want to put the player name near the start:
## all_p = all_p[[all_p.columns[-1]] + [*all_p.columns[:-1]]]

# (without the [*] notation you get an index):
# i.e.:  [*all_p.columns[:6]] ==> ['Gtm', '2B', '3B', 'AB', 'BA', 'BB']
# but all_p.columns[:6] ==> Index(['Gtm', '2B', '3B', 'AB', 'BA', 'BB'], dtype='object')

In [506]:
!ls -l player_game_dict.pkl

-rw-r--r--  1 ethanhaley  staff  1270893749 Sep  8 06:29 player_game_dict.pkl


In [508]:
!ls -l century_batting

-rw-r--r--  1 ethanhaley  staff  657772541 Sep  9 20:25 century_batting


So that's pretty nice, that the .csv dataframe only takes up half the space of the GB+ pickled dict version.  And now I can delete the pickled version, without having thrown out any info (actually I gained the names).

In [516]:
!rm player_game_dict.pkl